<a href="https://colab.research.google.com/github/opal16078/ML-hw1/blob/main/project1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [37]:
import pandas as pd
import numpy as np

df=pd.read_csv('house_data_train.csv', index_col = 0)
df = df.dropna()
# setting to read the data form 0 to index 16211
df1 = df.head(16211)

In [38]:
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn.preprocessing import PolynomialFeatures
from sklearn.impute import SimpleImputer
from sklearn.linear_model import LinearRegression , Lasso , Ridge
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold

#ignore warning 
def warn(*args, **kwargs):
    pass
import warnings
warnings.warn = warn

# adding feature
X = df1[['sqft_living' , 'sqft_lot','bedrooms','bathrooms','floors','waterfront','view','condition','grade','yr_built','yr_renovated','sqft_above', 'sqft_basement','sqft_living15' ,'sqft_lot15','lat','long']].values
y = df1['price'].values

# set the dimension of X
X = X.reshape(-1,17)

X_train , X_test , y_train , y_test = train_test_split(X,y, shuffle=False)
#splite the data to 5 set
kfold = KFold(n_splits = 5 , shuffle=True , random_state= 10)

steps = [("imp_mean", SimpleImputer()) , 
         ("scaler", StandardScaler()) , 
         ("transform", PolynomialFeatures(degree = 5,include_bias=False)) , 
        #  ("LinearReg" , LinearRegression())
         ("LassoReg" , Lasso(alpha = 50 , max_iter = 20000 , normalize = True))
        #  ("RidgeReg" , Ridge(alpha = 1 , max_iter = 20000 , normalize = True))
         ]

pipeline = Pipeline(steps)
pipeline.fit(X_train,y_train)
# rscore_test = pipeline.score(X_test,y_test)
# rscore_train = pipeline.score(X_train,y_train)
cv_score_train = cross_val_score( pipeline , X_train , y_train , cv=kfold )
cv_score_test = cross_val_score( pipeline , X_test , y_test , cv=kfold )
rscore_train = cv_score_train.mean()
rscore_test = cv_score_test.mean()
y_pred = pipeline.predict(X_test)

#finding mean squared error
mse = abs(cross_val_score( pipeline , X_train , y_train , cv=kfold , scoring ='neg_mean_squared_error' ).mean())
#finding mean absolute error
mae = abs(cross_val_score( pipeline , X_train , y_train , cv=kfold , scoring ='neg_mean_absolute_error' ).mean())



#  print('r2-score-test:', rscore_test.round(2) , '\nr2-score-train:', rscore_train.round(2))
print( "cv_score: ", cv_score_train ,'\nrscore-train:', rscore_train.round(4), 'rscore-test:', rscore_test.round(4),
         "\nmse: " , mse ,
         "\nmae: ", mae
      )

cv_score:  [0.80299569 0.79482245 0.79643747 0.80244429 0.85338763] 
rscore-train: 0.81 rscore-test: 0.8302 
mse:  -26174232194.04873 
mae:  91840.46059507599


In [39]:
import joblib

#export the pipeline to best model file
joblib.dump( pipeline ,'best_model.pkl')


['best_model.pkl']

In [40]:
# cv_score:  [0.76724335 0.79753651 0.79040556 0.76394899 0.80129259] ||||||||| d=3 alpha=50
# rscore-train: 0.7841 rscore-test: 0.793 
# mae:  98425.07753843135
# cv_score:  [0.80285515 0.80779197 0.80406324 0.77197231 0.82153559] ||||||||| d=4 alpha=50
# rscore-train: 0.8016 rscore-test: 0.7574 
# mae:  95367.37893774272
# cv_score:  [0.80299569 0.79482245 0.79643747 0.80244429 0.85338763]  |||||||||| d=5 alpha=100
# rscore-train: 0.81 rscore-test: 0.8302 
# mse:  -26174232194.04873 
# mae:  91840.46059507599

#some test of data